In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.A(html.Img(src ='data:image/png;base64,{}'.format(encoded_image.decode())), href='https://www.snhu.edu')),
    html.P('Created by Trever Andrew'),
    html.Hr(),
    html.Div([
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Label("Rescue Type"),
    dcc.RadioItems(
        id = 'filter-type',
        options = [{'label': 'Water Rescue', 'value': 'Water'},
                 {'label': 'Mountain & Wilderness Rescue', 'value': 'wilderness'},
                 {'label': 'Disaster & Tracking Rescue', 'value': 'disaster'},
                 {'label': 'Reset selection', 'value': 'reset'},
        ],
        value = 'reset',
        labelStyle = {'display': 'block'}
    ),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            #{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
            {"name": "Record Number", "id": "rec_num", "deletable": False, "selectable": True},
            {"name": "Age Upon Outcome", "id": "age_upon_outcome", "deletable": False, "selectable": True},
            {"name": "Animal ID", "id": "animal_id", "deletable": False, "selectable": True},
            {"name": "Animal type", "id": "animal_type", "deletable": False, "selectable": True},
            {"name": "Breed", "id": "breed", "deletable": False, "selectable": True},
            {"name": "Color", "id": "color", "deletable": False, "selectable": True},
            {"name": "Date of Birth", "id": "date_of_birth", "deletable": False, "selectable": True},
            {"name": "Date & time", "id": "datetime", "deletable": False, "selectable": True},
            {"name": "Month & Year", "id": "monthyear", "deletable": False, "selectable": True},
            {"name": "Name", "id": "name", "deletable": False, "selectable": True},
            {"name": "Outcome subtype", "id": "outcome_subtype", "deletable": False, "selectable": True},
            {"name": "Outcome", "id": "outcome_type", "deletable": False, "selectable": True},
            {"name": "Sex upon Outcome", "id": "sex_upon_outcome", "deletable": False, "selectable": True},
            {"name": "Latitude", "id": "location_lat", "deletable": False, "selectable": True},
            {"name": "Longitude", "id": "location_long", "deletable": False, "selectable": True},
            {"name": "Age upon outcome in weeks", "id": "age_upon_outcome_in_weeks", "deletable": False, "selectable": True},
        ],
        data=df.to_dict('records'),
        #Set up the features for your interactive data table to make it user-friendly for your client
        #page_auction sets up changeable pages
        page_action= "native",
        page_current= 0,
        page_size= 10,
        #below adds sorting and respective interabctale arrows for ascending and descending orders
        sort_action= "native",
        sort_mode= "multi",
        #Filtering allows you to search in the columns
        filter_action= "native",
        #selectable rows
        row_selectable= "single",
        selected_rows= [],
        
    ),

    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
#code to filter interactive data table with MongoDB queries  
        if filter_type == 'Water':
                query = {"breed": 
                            {"$regex": "Labrador Retriever Mix|Chesapeake Bay Retriever|Newfoundland", "$options": "i"}
                        }
                
        elif filter_type == 'wilderness':
                query = {"breed": 
                            {"$regex": "German Shepherd|Alaskan Malamute|Old English Sheepdog|Siberian Husky|Rottweiler" , "$options": "i"}
                        }
                
        elif filter_type == 'disaster':
                query = {"breed": 
                            {"$regex": "Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler" , "$options": "i"}
                        }                                 
                                   
        elif filter_type == 'reset':
                query = {}
            
        dff = pd.DataFrame.from_records(db.read(query))
            
        #this removes rows/columns that don't apply to the given filter    
        if '_id' in dff.columns:
            dff.drop(columns = ['_id'], inplace = True)
            
        return dff.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    #builds an empty chart if nothing is there
    if viewData is None or len(viewData) == 0:
        return[]
        
    dff = pd.DataFrame(viewData)
    
    fig = px.pie(
        dff,
        names='breed',
        title='Preferred Animals'
    )

    return [
        dcc.Graph(
            figure = fig,
            style = {'height': '1250px', 'width' : '1250px'}
        )
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '750px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://griffinpatient-milanalabama-3000.codio.io/proxy/8050/
